In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=eftJqPgA2maXRTwooAygvQrQwkgPTiaA-L4ziACPewY&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project reddit-master

Updated property [core/project].


In [2]:
!gsutil ls gs://reddit_final_results

gs://reddit_final_results/comments_posts_2018.csv
gs://reddit_final_results/comments_posts_2018.zip
gs://reddit_final_results/comments_posts_2018_V2.csv
gs://reddit_final_results/comments_posts_2018_V2.zip
gs://reddit_final_results/comments_posts_Fitness.pkl
gs://reddit_final_results/comments_posts_IAmA.pkl
gs://reddit_final_results/comments_posts_atheism.pkl
gs://reddit_final_results/comments_posts_aww.pkl
gs://reddit_final_results/comments_posts_europe.pkl
gs://reddit_final_results/comments_posts_funny.pkl
gs://reddit_final_results/comments_posts_gaming.pkl
gs://reddit_final_results/comments_posts_movies.pkl
gs://reddit_final_results/comments_posts_nba.pkl
gs://reddit_final_results/comments_posts_politics.pkl
gs://reddit_final_results/comments_posts_science.pkl
gs://reddit_final_results/comments_posts_technology.pkl
gs://reddit_final_results/comments_posts_todayilearned.pkl
gs://reddit_final_results/comments_posts_tokenized.csv
gs://reddit_final_results/comments_posts_tokenized.zip
g

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import joblib
import nltk
import ast
import logging


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble.forest import RandomForestClassifier
import datetime
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/pepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def loggg(msg):
    print("[INFO] {}: {}".format(datetime.datetime.now(), msg))

In [3]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [4]:
model_RFC_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits: 
    #!gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    loggg("comments_posts_" + subreddit + ".pkl downloaded")
    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")
    model_RFC_df = pd.concat([model_RFC_df, df], ignore_index=True, sort=True)


[INFO] 2019-12-01 16:13:14.018013: comments_posts_aww.pkl downloaded
[INFO] 2019-12-01 16:13:15.436744: comments_posts_nba.pkl downloaded
[INFO] 2019-12-01 16:13:17.100632: comments_posts_movies.pkl downloaded
[INFO] 2019-12-01 16:13:19.296291: comments_posts_todayilearned.pkl downloaded
[INFO] 2019-12-01 16:13:21.342605: comments_posts_IAmA.pkl downloaded
[INFO] 2019-12-01 16:13:24.011856: comments_posts_Fitness.pkl downloaded
[INFO] 2019-12-01 16:13:27.211075: comments_posts_worldnews.pkl downloaded
[INFO] 2019-12-01 16:13:30.583880: comments_posts_technology.pkl downloaded
[INFO] 2019-12-01 16:13:32.807254: comments_posts_europe.pkl downloaded
[INFO] 2019-12-01 16:13:36.618931: comments_posts_politics.pkl downloaded
[INFO] 2019-12-01 16:13:38.753457: comments_posts_atheism.pkl downloaded
[INFO] 2019-12-01 16:13:43.345518: comments_posts_science.pkl downloaded
[INFO] 2019-12-01 16:13:46.013548: comments_posts_funny.pkl downloaded
[INFO] 2019-12-01 16:13:47.815133: comments_posts_gami

In [5]:
X = model_RFC_df['body']
y = model_RFC_df['subreddit_id']

def train_dev_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    return dict(X_train=X_train, 
                X_test=X_test, 
                y_train=y_train.astype('int'), 
                y_test=y_test.astype('int'))

loggg(model_RFC_df.shape)

[INFO] 2019-12-01 16:14:02.816284: (10833350, 4)


In [6]:
model_RFC_df_split = train_dev_test_split(X,y)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=" ".join, ngram_range=(1,1), max_df=0.5)),
    ("clf", RandomForestClassifier(min_samples_leaf=1,max_depth=30,n_estimators=500,n_jobs=4))
    ])

In [7]:
pipeline.fit(model_RFC_df_split['X_train'], model_RFC_df_split['y_train'])

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<built-in method join of str object at 0x7feca9deb970>,
                                 smooth_idf=True, stop_words=None,
                                 strip_acce...
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=30,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                     

In [8]:
predictions = pipeline.predict(model_RFC_df_split['X_test'])
print(classification_report(model_RFC_df_split['y_test'], predictions))

              precision    recall  f1-score   support

           0       0.58      0.75      0.65    232289
           1       0.44      0.09      0.15    232243
           2       0.51      0.50      0.50    232080
           3       0.20      0.76      0.32    232145
           4       0.50      0.42      0.46    232007
           5       0.13      0.19      0.16    231810
           6       0.53      0.37      0.44    232122
           7       0.59      0.46      0.51    232173
           8       0.63      0.61      0.62    232026
           9       0.47      0.41      0.44    232104
          10       0.39      0.39      0.39    232721
          11       0.45      0.36      0.40    232131
          12       0.32      0.03      0.06    231997
          13       0.43      0.12      0.19    232157

    accuracy                           0.39   3250005
   macro avg       0.44      0.39      0.38   3250005
weighted avg       0.44      0.39      0.38   3250005



In [9]:
joblib.dump(pipeline, 'model_rfc.joblib')

['model_rfc.joblib']

In [10]:
!gsutil cp model_rfc.joblib gs://reddit_models

Copying file://model_rfc.joblib [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][588.4 MiB/588.4 MiB]    4.8 MiB/s                                   
Operation completed over 1 objects/588.4 MiB.                                    
